# Create Axolotl Docker images

At its simples, finetuning a model requires model weights, and a dataset. Although simple conceptually, several questions presents themselves when fine tuning a model:

1. Are you starting with a completion model, and fine tuning it further on completion data, such as in the case of Domain Adaptation?
2. Are you starting with a completion model, and fine tuning it further with instructions?
3. How will you tune the instructions to reflect your user base queries the most appropriately?
4. If continually fine tuning, how will you assess your models against catastrophic forgetting?

In addition to conceptual questions, several other engineering ones present themselves.
1. Will you use LoRA, QLoRA, or any other form of optimization?
2. Is your job parallel in nature, and requires distribution?
3. Do you need to register your model in MLFlow?

To help ease the configuration and execution of finetuning jobs, several frameworks are being developed. One of these frameworks is [Axolotl](https://github.com/OpenAccess-AI-Collective/axolotl) - a tool to streamline the fine-tuning of various AI models, offering support for multiple configurations and architectures. Companies often find themselves running hundreds of finetuning experiments in parallel.

## Goal

In this notebook, you will create a new curated environment to run Axolotl on Azure.
While you only need 1 environment, you will create it twice, for the sake of exploring two different methods to create the environment.
When building a docker image to Azure, you will have to decide on a design question around the source of your Docker image. In this instance, you will create a new curated environment based on the Axolotl gold image.

In a future instance, you will install Axolotl on an ACPT image. The latter will enable you to leverage features such as [Nebula](https://learn.microsoft.com/en-us/azure/machine-learning/reference-checkpoint-performance-for-large-models?view=azureml-api-2&tabs=PYTORCH). Nebula is a framework that will enable you to book checkpoint speeds and reduce costs.

###  A | Create a curated environment based on the Axolotl Gold image

In the first section of this guide, you will create an environment called "axolotl_gold", which is the first option of Docker images. This image is based off the Axolotl gold Image, and is modified to include `azureml-mlflow`.

The definition of this simple image will be stored in a DockerFile. The creation of the environment will be performed through the SDK. You only need to run this notebook once to create the environment.

Make sure the Jupyter Kernel you select has the Azure ML SDK v2 installed.

In [1]:
import azureml.core
workspace = azureml.core.Workspace.from_config()
workspace

Workspace.create(name='aml-au', subscription_id='68092087-0161-4fb5-b51d-32f18ac56bf9', resource_group='aml-au')

The code snippet below will build a new environment in your workspace, assuming the environment doesn't exist yet.
The environment will be built based on the docker context, more specifically the contents of the file `src/docker_gold/dockerfile`. As you can see from inspecting this file, this image will be simply based on the master image `winglian/axolotl-cloud:main-latest`.
If you wonder where this image name is from: it's from the Axolotl website please [see this link](https://github.com/OpenAccess-AI-Collective/axolotl?tab=readme-ov-file#cloud-gpu).

The resulting image will work on Azure, but since it is built from scratch, it will not include any other library found in ACPT.
The process will take approximately 12 minutes. During this time, you can see the progress of the build job in the portal by clicking on the link displayed beneath the Jupyter cell.

In [ ]:
environment_display_name = 'axolotl_gold'
all_environments_in_my_workspace = azureml.core.Environment.list(workspace).keys()
if environment_display_name not in all_environments_in_my_workspace:
    print(f"Building {environment_display_name} environment")
    environment = azureml.core.Environment.from_docker_build_context(
        name = environment_display_name,
        docker_build_context=azureml.core.environment.DockerBuildContext.from_local_directory(
            workspace=workspace,
            path='src/docker_gold',
            dockerfile_path='dockerfile'
        )
    )
    environment = environment.register(workspace).build(workspace).wait_for_completion()
else:
    print(f"Environment {environment_display_name} already exists, using it.")
environment = azureml.core.Environment.get(workspace=workspace, name=environment_display_name)

### B | Axolotl installed on ACPT

An alternative method is to install Axolotl on ACPT. Axolotl requires Pytorch >2 and Python > 3.10.

The latest ACPT (beta) mcr.microsoft.com/aifx/acpt/stable-ubuntu2004-cu118-py310-torch220 ships with:
- Ubuntu 20.04
- CUDA 11.8
- Pytorch 2.2.0
- DeepSpeed 0.13.4
- Nebula 0.16


In [ ]:
environment_display_name = 'axolotl_acpt'
all_environments_in_my_workspace = azureml.core.Environment.list(workspace).keys()
if environment_display_name not in all_environments_in_my_workspace:
    print(f"Building {environment_display_name} environment")
    environment = azureml.core.Environment.from_docker_build_context(
        name = environment_display_name,
        docker_build_context=azureml.core.environment.DockerBuildContext.from_local_directory(
            workspace=workspace,
            path='src/docker_axolotl_acpt',
            dockerfile_path='dockerfile'
        )
    )
    environment = environment.register(workspace).build(workspace).wait_for_completion()
else:
    print(f"Environment {environment_display_name} already exists, using it.")
environment = azureml.core.Environment.get(workspace=workspace, name=environment_display_name)

This concludes the first notebook - you should now have up to two environments created, both with Axolotl. The 2nd image is based on ACPT (which contains Nebula) and therefore will be used in the remainder of the notebooks.